In [ ]:
import numpy as np 
import pandas as pd 
import itertools
from pandasql import sqldf

In [2]:
pd.options.display.max_rows=None

In [3]:
df_us_counties = pd.read_csv('data/us-counties.csv')

In [4]:
#sqldf("SELECT * FROM df_us_counties WHERE county = 'Snohomish'")

In [5]:
df_us_counties = sqldf("SELECT * FROM df_us_counties ORDER BY county, state, date")

In [6]:
df_us_counties[:100]

,date,county,state,fips,cases,deaths
0,2020-03-19,Abbeville,South Carolina,45001.0,1,0
1,2020-03-20,Abbeville,South Carolina,45001.0,1,0
2,2020-03-21,Abbeville,South Carolina,45001.0,1,0
3,2020-03-22,Abbeville,South Carolina,45001.0,1,0
4,2020-03-23,Abbeville,South Carolina,45001.0,1,0
5,2020-03-24,Abbeville,South Carolina,45001.0,1,0
6,2020-03-25,Abbeville,South Carolina,45001.0,3,0
7,2020-03-26,Abbeville,South Carolina,45001.0,4,0
8,2020-03-27,Abbeville,South Carolina,45001.0,4,0
9,2020-03-28,Abbeville,South Carolina,45001.0,4,0


In [7]:
# code to convert cumulative deaths and cases into daily new deaths and cases
daily_deaths = []
daily_cases = []
county = df_us_counties.iloc[0]['county']
state = df_us_counties.iloc[0]['state']
i = 0
daily_deaths.append(df_us_counties.iloc[0]['deaths'])
daily_cases.append(df_us_counties.iloc[0]['cases'])
pdc = daily_deaths[0]
pcc = daily_cases[0]
for d in df_us_counties.iloc[1:].iterrows():
    
    if d[1]['county'] == county and d[1]['state'] == state:
        daily_deaths.append(max(d[1]['deaths'] - pdc, 0))
        daily_cases.append(max(d[1]['cases'] - pcc, 0))
        pdc = d[1]['deaths']
        pcc = d[1]['cases']
    else:
        daily_deaths.append(d[1]['deaths'])
        daily_cases.append(d[1]['deaths'])
        pdc = 0
        pcc = 0
        county = d[1]['county']
        state = d[1]['state']
    i += 1

In [10]:
df_us_counties['daily_deaths'] = daily_deaths
df_us_counties['daily_cases'] = daily_cases

In [11]:
df_us_counties['daily_ratio'] = df_us_counties.daily_deaths / df_us_counties.daily_cases
df_us_counties = df_us_counties.fillna(0)
df_us_counties = df_us_counties.replace(np.inf, 0)

In [12]:
df_us_counties[:200]

,date,county,state,fips,cases,deaths,daily_deaths,daily_cases,daily_ratio
0,2020-03-19,Abbeville,South Carolina,45001.0,1,0,0,1,0.000000
1,2020-03-20,Abbeville,South Carolina,45001.0,1,0,0,0,0.000000
2,2020-03-21,Abbeville,South Carolina,45001.0,1,0,0,0,0.000000
3,2020-03-22,Abbeville,South Carolina,45001.0,1,0,0,0,0.000000
4,2020-03-23,Abbeville,South Carolina,45001.0,1,0,0,0,0.000000
5,2020-03-24,Abbeville,South Carolina,45001.0,1,0,0,0,0.000000
6,2020-03-25,Abbeville,South Carolina,45001.0,3,0,0,2,0.000000
7,2020-03-26,Abbeville,South Carolina,45001.0,4,0,0,1,0.000000
8,2020-03-27,Abbeville,South Carolina,45001.0,4,0,0,0,0.000000
9,2020-03-28,Abbeville,South Carolina,45001.0,4,0,0,0,0.000000


In [21]:
# only include those counties that have had at least one day with at 1000+ cases 
plus_1000 = sqldf("SELECT distinct county, state, daily_cases FROM df_us_counties WHERE daily_cases > 1000 ORDER BY daily_ratio DESC")

In [22]:
len(plus_1000)

119

In [23]:
# get a list of 

county_state = []

for r in plus_1000.iterrows():
    county_state.append((r[1]['county'], r[1]['state']))

#county_state


In [27]:
data = sqldf("SELECT * FROM df_us_counties WHERE county = '" + county_state[1][0] + "' AND state = '" + county_state[1][1] + "' ORDER BY state, date")
len(data)

72

In [32]:
training_data = []
result_row = []
data_row = []

k = 0
for cs in county_state:

    # limit it for speed
    if k >= 10:
        break
    k += 1
    
    data = sqldf("SELECT * FROM df_us_counties WHERE county = '" + cs[0] + "' AND state = '" + cs[1] + "' ORDER BY state, date")
        
    for i in range(10, len(data)+1):
        data_row = []
        for j in range(i-10, i):
            data_row.append((data.iloc[j]['daily_cases'], data.iloc[j]['daily_deaths']))
        training_data.append(list(itertools.chain(*data_row)))
        result_row.append(data.iloc[i-1]['daily_ratio'])

In [34]:
len(training_data)

626

In [36]:
# run this cell to see the input and result vectors for the ML input
i = 0
for t in training_data:
    print(i, t, result_row[i])
    i += 1
    
    if i > 100:
        break

0 [0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 1, 0, 7, 0, 2, 0, 6, 0, 17, 0] 0.0
1 [1, 0, 1, 0, 0, 0, 2, 0, 1, 0, 7, 0, 2, 0, 6, 0, 17, 0, 15, 0] 0.0
2 [1, 0, 0, 0, 2, 0, 1, 0, 7, 0, 2, 0, 6, 0, 17, 0, 15, 0, 44, 0] 0.0
3 [0, 0, 2, 0, 1, 0, 7, 0, 2, 0, 6, 0, 17, 0, 15, 0, 44, 0, 59, 0] 0.0
4 [2, 0, 1, 0, 7, 0, 2, 0, 6, 0, 17, 0, 15, 0, 44, 0, 59, 0, 114, 1] 0.008771929824561403
5 [1, 0, 7, 0, 2, 0, 6, 0, 17, 0, 15, 0, 44, 0, 59, 0, 114, 1, 61, 4] 0.06557377049180328
6 [7, 0, 2, 0, 6, 0, 17, 0, 15, 0, 44, 0, 59, 0, 114, 1, 61, 4, 134, 2] 0.014925373134328358
7 [2, 0, 6, 0, 17, 0, 15, 0, 44, 0, 59, 0, 114, 1, 61, 4, 134, 2, 182, 4] 0.02197802197802198
8 [6, 0, 17, 0, 15, 0, 44, 0, 59, 0, 114, 1, 61, 4, 134, 2, 182, 4, 698, 14] 0.02005730659025788
9 [17, 0, 15, 0, 44, 0, 59, 0, 114, 1, 61, 4, 134, 2, 182, 4, 698, 14, 1133, 6] 0.00529567519858782
10 [15, 0, 44, 0, 59, 0, 114, 1, 61, 4, 134, 2, 182, 4, 698, 14, 1133, 6, 1942, 23] 0.01184346035015448
11 [44, 0, 59, 0, 114, 1, 61, 4, 134, 2, 182, 4, 698, 

In [37]:
# now we have data and result vectors to train a ML model 

In [38]:
from sklearn.ensemble import RandomForestRegressor

In [39]:
ml = RandomForestRegressor(n_estimators=100, max_depth=10,random_state=0)

In [40]:
ml.fit(training_data, result_row)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [41]:
# look through predicted and actual daily death:case ratios on each vector
for i in range(len(training_data)):
    print(i, ml.predict([training_data[i]])[0], result_row[i])
    i += 1

0 0.0 0.0
1 0.0 0.0
2 0.0 0.0
3 0.0 0.0
4 0.008771929824561424 0.008771929824561403
5 0.06557377049180334 0.06557377049180328
6 0.014914032556032599 0.014925373134328358
7 0.02197802197802194 0.02197802197802198
8 0.020047469307240252 0.02005730659025788
9 0.005273274283863604 0.00529567519858782
10 0.011647464603069898 0.01184346035015448
11 0.011603677239906619 0.01162147205312673
12 0.015395097298479925 0.01549841493483621
13 0.011302031231397686 0.011029411764705883
14 0.028694711590870333 0.028691660290742157
15 0.019536796884758005 0.019538882375928098
16 0.03364306312079456 0.03364306312079462
17 0.05385876120923767 0.05371723248818221
18 0.04897693490771174 0.048983364140480594
19 0.0540035445017144 0.054177545691906005
20 0.04805291137983809 0.048041894353369766
21 0.0531349731322566 0.05308185883725452
22 0.08479586938862683 0.0848498635122839
23 0.0679297580017157 0.06776638430673206
24 0.08952531476686293 0.08951452806352644
25 0.09046995457393908 0.09047844623401231
26 0.0

238 0.03516998827667053 0.035169988276670575
239 0.192135835567471 0.19213583556747096
240 0.10038953968404711 0.09877488514548238
241 0.10865786931595972 0.10825545171339564
242 0.18870523415977986 0.1887052341597796
243 0.07855488921988897 0.07859798194370685
244 0.04619782114077925 0.045877659574468085
245 0.07570196831338921 0.07569721115537849
246 0.11085409706150288 0.11278952668680765
247 0.1429380080418251 0.14269662921348314
248 0.0 0.0
249 0.0 0.0
250 0.0 0.0
251 0.0 0.0
252 0.008771929824561424 0.008771929824561403
253 0.06557377049180334 0.06557377049180328
254 0.014914032556032599 0.014925373134328358
255 0.02197802197802194 0.02197802197802198
256 0.020047469307240252 0.02005730659025788
257 0.005273274283863604 0.00529567519858782
258 0.011647464603069898 0.01184346035015448
259 0.011603677239906619 0.01162147205312673
260 0.015395097298479925 0.01549841493483621
261 0.011302031231397686 0.011029411764705883
262 0.028694711590870333 0.028691660290742157
263 0.01953679688

469 0.12267839687194546 0.12267839687194526
470 0.14030051531233093 0.14002745636399294
471 0.17223133813936845 0.172020175205734
472 0.1938359653591441 0.19383596535914416
473 0.08839296845779822 0.08839296845779827
474 0.10925553319919537 0.10925553319919518
475 0.03972602739726036 0.03972602739726028
476 0.03161274388737964 0.0316127438873796
477 0.36799276672694325 0.3679927667269439
478 0.17559580076344505 0.17602548785344485
479 0.16482220259533154 0.1646661665416354
480 0.13765432098765415 0.13765432098765432
481 0.1135593220338985 0.1135593220338983
482 0.08246465728076197 0.08266722759781236
483 0.08246465728076197 0.08218029350104822
484 0.10446804014964822 0.10445468509984639
485 0.14200595388321843 0.14212982605006363
486 0.1902173913043477 0.19021739130434784
487 0.1484004790514484 0.14827455441789913
488 0.07964443809910036 0.07969924812030076
489 0.05679781957325583 0.0568234746157429
490 0.03516998827667053 0.035169988276670575
491 0.192135835567471 0.19213583556747096


In [42]:
# use follow up on a few of the outliers, they look ok
sqldf("SELECT * FROM df_us_counties WHERE daily_ratio = 0.29411764705882354")

,date,county,state,fips,cases,deaths,daily_deaths,daily_cases,daily_ratio
0,2020-04-22,Chaffee,Colorado,8015.0,55,9,5,17,0.294118
1,2020-05-07,Cumberland,Pennsylvania,42041.0,403,31,5,17,0.294118
2,2020-03-27,DuPage,Illinois,17043.0,199,6,5,17,0.294118
3,2020-05-05,Gloucester,New Jersey,34015.0,1467,61,10,34,0.294118
4,2020-05-10,Hampshire,Massachusetts,25015.0,676,55,5,17,0.294118
5,2020-05-11,New Haven,Connecticut,9009.0,9260,716,15,51,0.294118
6,2020-05-07,Oakland,Michigan,26125.0,7624,789,15,51,0.294118
7,2020-03-28,Santa Clara,California,6085.0,591,26,5,17,0.294118


In [45]:
# predict a single vector
print(ml.predict([[45, 10, 130, 4, 101, 7, 141, 8, 101, 1, 130, 11, 112, 7, 82, 6, 219, 10, 251, 16]]))

[0.0416305]


In [46]:
# looks like the current daily number of cases and deaths are driving most of the per
# previous 9 days the remaining 60-70%
ml.feature_importances_

array([0.03558571, 0.06344726, 0.01604192, 0.02063547, 0.02011346,
       0.09819835, 0.0034646 , 0.00600672, 0.00505553, 0.00607164,
       0.00330397, 0.00322439, 0.00441759, 0.00430062, 0.01635296,
       0.00647894, 0.00778904, 0.00570081, 0.05324479, 0.62056623])